In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import seaborn as sbs
import matplotlib.pyplot as plt

In [3]:
df_tickers = pd.read_excel('tickers/tickers_only.xlsx')

In [12]:
apple = yf.Ticker(df_tickers['Ticker'].values[1])

In [18]:
apple_history = apple.history(start='2000-01-01', end='2024-01-26')

In [20]:
apple_history.to_csv('apple_history.csv')